In [1]:
import pandas as pd #importing the Pandas Library as 'pd'
from keplergl import KeplerGl #importing KeplerGl
import geopandas as gpd #importing geopandas as 'gpd'

In [2]:

map = KeplerGl(height=600, width=800)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [9]:
filename = "Liverpool.geojson"
file = open(filename)
df = gpd.read_file(file)

In [10]:
map.add_data(data=df, name="Construction")

In [16]:
map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'mefqat9', 'type': …

In [17]:
map.save_to_html(file_name='Liverpool.html',read_only=True)

Map saved to Liverpool.html!
